In [ ]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,313 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,021 kB

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-03-18 19:14:02--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.58MB/s    in 0.6s    

2023-03-18 19:14:04 (1.58 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...|2015-08-31 00:00:00|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...| 

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3ARRMDEGED8RD|          5|            0|          0|   N|                Y|
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
| RUXJRZCT6953M|          1|            2|          2|   N|                N|
| R7EO0UO6BPB71|          1|            0|          0|   N|                Y|
|R39NJY2YJ1JFSV|          5|            0|          0|   N|                Y|
|R31SR7REWNX7CF|          5|            0|          0|   N|                Y|
| RVBP8I1R0CTZ8|          3|            1|          2|   N|                Y|
|R1QF6RS1PDLU18|          4|            1|          1|   N|                Y|
|R23AICGEDAJQL1|          1|            0|          0|   N|                Y|
|R2EY3N4K9W19UP|          5|            3|          4|   N|     

In [ ]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 
# to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.
votes_df = vine_df.filter("total_votes >= 20")

votes_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R35OW0SJPJWAHO|          1|           12|        162|   N|                Y|
| RAP2ETU511BOI|          1|            5|         99|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
|R1RMOCWMV8F4VH|          1|            5|         21|   N|                Y|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|
|R38OLNY5GR1PW6|          5|           44|         44|   N|     

In [ ]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve 
# all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
filtered_votes_df = votes_df.filter("helpful_votes / total_votes >= 0.50")

filtered_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|
|R38OLNY5GR1PW6|          5|           44|         44|   N|                Y|
|R2GK7RJNIAREIT|          1|           30|         32|   N|                Y|
|R2OWRP6FXXECMK|          1|           42|         44|   N|                Y|
| ROR2FE6Y8EZYG|          5|           19|         22|   N|     

In [ ]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or 
# table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
paid_vine_df = filtered_votes_df.filter("vine == 'Y'")
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3J2DPYUH6453D|          5|          136|        153|   Y|                N|
|R25QTFLERWK6SJ|          3|          852|        956|   Y|                N|
|R1BG71K9B61LI1|          4|           39|         39|   Y|                N|
| R8WGX426U56E0|          5|           19|         21|   Y|                N|
| R614TCIDD36LE|          4|          131|        147|   Y|                N|
|R28X5K1OZBQSBK|          5|           25|         28|   Y|                N|
|R2CUDWKAUWGHIT|          5|           19|         24|   Y|                N|
| R886ADSI1HBU5|          5|           88|         97|   Y|                N|
|R3TTJTAND1CCSV|          5|           21|         23|   Y|                N|
|R1W39WXYWNFJBG|          5|           32|         35|   Y|     

In [42]:
#get total of paid_vine_df

total_paid_df = paid_vine_df.count()
total_paid_df


fivestar_paid_df = paid_vine_df.filter("star_rating == '5'").count()

fivestar_paid_df


paid_5star_df = (fivestar_paid_df / total_paid_df) * 100
paid_5star_df = spark.createDataFrame([(paid_5star_df,)], ['Percentage of 5-Star Reviews'])
paid_5star_df.show()

+----------------------------+
|Percentage of 5-Star Reviews|
+----------------------------+
|           44.11267605633803|
+----------------------------+



In [44]:
total_paid_df

1775

In [45]:
fivestar_paid_df

783

In [47]:
# Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
unpaid_vine_df = filtered_votes_df.filter("vine == 'N'")

unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|
|R38OLNY5GR1PW6|          5|           44|         44|   N|                Y|
|R2GK7RJNIAREIT|          1|           30|         32|   N|                Y|
|R2OWRP6FXXECMK|          1|           42|         44|   N|                Y|
| ROR2FE6Y8EZYG|          5|           19|         22|   N|     

In [48]:

total_unpaid_df = unpaid_vine_df.count()
total_unpaid_df


fivestar_unpaid_df = unpaid_vine_df.filter("star_rating == '5'").count()

fivestar_unpaid_df

unpaid_5star_df = (fivestar_unpaid_df / total_unpaid_df) * 100
unpaid_5star_df = spark.createDataFrame([(unpaid_5star_df,)], ['Percentage of 5-Star Reviews'])
unpaid_5star_df.show()

+----------------------------+
|Percentage of 5-Star Reviews|
+----------------------------+
|          46.454739290560184|
+----------------------------+



In [49]:
total_unpaid_df

77385

In [50]:
fivestar_unpaid_df

35949